In [1]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
from common import sanitize
# import torch.nn as nn
# torch.nn.modules.module.Module._call_impl = _call_impl2
# torch.nn.Module._call_impl=_call_impl2
# nn.Module._call_impl=_call_impl2


In [2]:

import time
from transformers import AutoModel
from transformers import AutoConfig
import torch
from torch import nn
torch.set_grad_enabled(False)

In [85]:
from calibrate_e_ml import get_module_info
from calibrate_e_ml import print_info

from cg.node import construct_aggregation_graph

In [4]:
seq_len = 100
bs = 4
input_ids = torch.randint(1000, size=(bs, seq_len), dtype=torch.long, device='cpu')
cuda_exist = torch.cuda.is_available()
device = torch.device("cuda" if cuda_exist else "cpu")

In [5]:
model_name = "prajjwal1/bert-tiny"
information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(information)


BertEmbeddings 1
<class 'dict'> embeddings:BertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

BertSelfAttention 2
<class 'tuple'> encoder.layer.0.attention.self:BertSelfAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

BertSelfOutput 2
<class 'tuple'> encoder.layer.0.attention.output:BertSelfOutput [torch.Size([4, 100, 128]), torch.Size([4, 100, 128])]

BertAttention 2
<class 'dict'> encoder.layer.0.attention:BertAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:BertAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None]

BertIntermediate 2
<class 'tuple'> encoder.layer.0.intermediate:BertIntermediate [torch.Size([4, 100, 128])]

BertOutput 2
<class 'tuple'> encoder.layer.0.output:BertOutput [torch.Size([4, 100, 512]), torch.Size([4, 100, 128])]

BertLaye

In [6]:
import inspect
from run_level_exp import get_model_flops_mem_bytes, wrapped_partial

In [7]:
sel_attn = information['BertSelfAttention'][0]
fn = sel_attn['module']
fi = sel_attn['inputs']
fi_kwargs = sel_attn['in_kwargs']

fn_fwd = fn.forward
#  unify fi and fi_kwargs
fn_args = inspect.getfullargspec(fn_fwd).args
fill_args = dict()
fi_args = fn_args[1:1 + len(fi)]
ti = []
for fi_k, fi_v in zip(fi_args, fi):
    if isinstance(fi_v, torch.Tensor):
        ti.append(fi_v)
    else:
        fill_args[fi_k] = fi_v
for k, v in fi_kwargs.items():
    if isinstance(v, torch.Tensor):
        ti.append(v)
    else:
        fill_args[k] = v
# wrap forward into traceable fn (only tensor args)
# https://github.com/pytorch/pytorch/issues/14455#issuecomment-445962680
fn.forward = wrapped_partial(fn.forward, **fill_args)

In [8]:
flops, mem_bytes = get_model_flops_mem_bytes(fn, ti, 'BertEncoder')
print(flops/1e6, mem_bytes/1024/1024)

60.5152 6.62158203125


In [27]:
model_name = "distilbert-base-uncased"
distilbert_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(distilbert_information)

Embeddings 1
<class 'tuple'> embeddings:Embeddings [torch.Size([4, 100])]

MultiHeadSelfAttention 6
<class 'dict'> transformer.layer.0.attention:MultiHeadSelfAttention {'query': torch.Size([4, 100, 768]), 'key': torch.Size([4, 100, 768]), 'value': torch.Size([4, 100, 768]), 'mask': torch.Size([4, 100]), 'head_mask': None, 'output_attentions': False}

FFN 6
<class 'tuple'> transformer.layer.0.ffn:FFN [torch.Size([4, 100, 768])]

TransformerBlock 6
<class 'dict'> transformer.layer.0:TransformerBlock {'x': torch.Size([4, 100, 768]), 'attn_mask': torch.Size([4, 100]), 'head_mask': None, 'output_attentions': False}

Transformer 1
<class 'dict'> transformer:Transformer {'x': torch.Size([4, 100, 768]), 'attn_mask': torch.Size([4, 100]), 'head_mask': [None, None, None, None, None, None], 'output_attentions': False, 'output_hidden_states': False, 'return_dict': False}



In [42]:
get_non_parametric_ml_op(distilbert_information)

Embeddings,embeddings:Embeddings

MultiHeadSelfAttention,transformer.layer.0.attention:MultiHeadSelfAttention
		OpNode(id=scores, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=q, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=53, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=scores, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.3, op=aten::softmax, flops=2400000, mem_bytes=5760000, 
			inputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=64, dtype=int)
				DataNode(id=65, dtype=None)],
			outputs=[
				DataNode(id=input.3, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=x, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=weights, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=v, dtype=Float, shape=[4, 12, 100, 64])],
			outputs=[
				DataNode(id=x, dtype=Float, shape=[4, 12, 100, 64])],)

MultiHeadSelfAttention,transformer.layer.1.at

/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/modeling_utils.py:1759: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert all(



FFN,transformer.layer.1.ffn:FFN

FFN,transformer.layer.2.ffn:FFN

FFN,transformer.layer.3.ffn:FFN

FFN,transformer.layer.4.ffn:FFN

FFN,transformer.layer.5.ffn:FFN

TransformerBlock,transformer.layer.0:TransformerBlock
		OpNode(id=input.2, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=attention, 
			inputs=[
				DataNode(id=input.1, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=26, dtype=int)
				DataNode(id=19, dtype=None)],
			outputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])],)

TransformerBlock,transformer.layer.1:TransformerBlock
		OpNode(id=input.2, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=attention, 
			inputs=[
				DataNode(id=input.1, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=26, dtype=int)
				DataNode(id=19, dtype=None)],
			outputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])],)

TransformerBlock,transformer.layer.2:TransformerBlock
		OpNode(id=input.2, op=aten::softmax, flops=24000

In [26]:
model_name = "bert-base-uncased"
bert_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(bert_information)

BertEmbeddings 1
<class 'dict'> embeddings:BertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

BertSelfAttention 12
<class 'tuple'> encoder.layer.0.attention.self:BertSelfAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

BertSelfOutput 12
<class 'tuple'> encoder.layer.0.attention.output:BertSelfOutput [torch.Size([4, 100, 768]), torch.Size([4, 100, 768])]

BertAttention 12
<class 'dict'> encoder.layer.0.attention:BertAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:BertAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None]

BertIntermediate 12
<class 'tuple'> encoder.layer.0.intermediate:BertIntermediate [torch.Size([4, 100, 768])]

BertOutput 12
<class 'tuple'> encoder.layer.0.output:BertOutput [torch.Size([4, 100, 3072]), torch.Size([4, 100, 768])]

Ber

In [36]:
get_non_parametric_ml_op(bert_information['BertSelfAttention'])

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=query_layer, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=66, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input, op=aten::softmax, flops=2400000, mem_bytes=5760000, 
			inputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=72, dtype=int)
				DataNode(id=73, dtype=None)],
			outputs=[
				DataNode(id=input, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=context_layer.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=attention_probs, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=value_layer, dtype=Float, shape=[4, 12, 100, 64])],
			outputs=[
				DataNode(id=context_layer.1, dtype=Float, shape=[4, 12, 100, 64])],)


In [25]:
model_name = "roberta-base"
roberta_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(roberta_information)

RobertaEmbeddings 1
<class 'dict'> embeddings:RobertaEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

RobertaSelfAttention 12
<class 'tuple'> encoder.layer.0.attention.self:RobertaSelfAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

RobertaSelfOutput 12
<class 'tuple'> encoder.layer.0.attention.output:RobertaSelfOutput [torch.Size([4, 100, 768]), torch.Size([4, 100, 768])]

RobertaAttention 12
<class 'dict'> encoder.layer.0.attention:RobertaAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:RobertaAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None]

RobertaIntermediate 12
<class 'tuple'> encoder.layer.0.intermediate:RobertaIntermediate [torch.Size([4, 100, 768])]

RobertaOutput 12
<class 'tuple'> encoder.layer.0.output:RobertaOutput [torch.Size([4, 100, 

In [31]:
get_non_parametric_ml_op(roberta_information['RobertaSelfAttention'])

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=query_layer, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=66, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input, op=aten::softmax, flops=2400000, mem_bytes=5760000, 
			inputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=72, dtype=int)
				DataNode(id=73, dtype=None)],
			outputs=[
				DataNode(id=input, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=context_layer.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=attention_probs, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=value_layer, dtype=Float, shape=[4, 12, 100, 64])],
			outputs=[
				DataNode(id=context_layer.1, dtype=Float, shape=[4, 12, 100, 64])],)


In [24]:
# for k, info in information.items():
#     for infoi in info:
#         ii = infoi['inputs']
#         if isinstance(ii, dict):
#             print(type(ii), infoi['name'],  {iik:v.shape if isinstance(v, torch.Tensor) else v for iik, v in ii.items()})
#         elif isinstance(ii, tuple):
#             print(type(ii), infoi['name'],  [v.shape if isinstance(v, torch.Tensor) else v for v in ii])
#         else:
#             print(type(ii), 'todo', k, infoi)
#         print()
model_name = "google/mobilebert-uncased"
mobilebert_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(mobilebert_information)

NoNorm 193
<class 'tuple'> embeddings.LayerNorm:NoNorm [torch.Size([4, 100, 512])]

MobileBertEmbeddings 1
<class 'dict'> embeddings:MobileBertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None}

BottleneckLayer 48
<class 'tuple'> encoder.layer.0.bottleneck.input:BottleneckLayer [torch.Size([4, 100, 512])]

Bottleneck 24
<class 'tuple'> encoder.layer.0.bottleneck:Bottleneck [torch.Size([4, 100, 512])]

MobileBertSelfAttention 24
<class 'tuple'> encoder.layer.0.attention.self:MobileBertSelfAttention [torch.Size([4, 100, 128]), torch.Size([4, 100, 128]), torch.Size([4, 100, 512]), torch.Size([4, 1, 1, 100]), None, False]

MobileBertSelfOutput 24
<class 'tuple'> encoder.layer.0.attention.output:MobileBertSelfOutput [torch.Size([4, 100, 128]), torch.Size([4, 100, 128])]

MobileBertAttention 24
<class 'dict'> encoder.layer.0.attention:MobileBertAttention {'output_attentions': False}
<class 'tuple'> encoder.layer

In [35]:
get_non_parametric_ml_op(mobilebert_information['MobileBertSelfAttention'])

		OpNode(id=attention_scores.1, op=aten::matmul, flops=10240000, mem_bytes=1049600, 
			inputs=[
				DataNode(id=query_layer, dtype=Float, shape=[4, 4, 100, 32])
				DataNode(id=68, dtype=Float, shape=[4, 4, 32, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 4, 100, 100])],)
		OpNode(id=input, op=aten::softmax, flops=800000, mem_bytes=1920000, 
			inputs=[
				DataNode(id=input.4, dtype=Float, shape=[4, 4, 100, 100])
				DataNode(id=74, dtype=int)
				DataNode(id=75, dtype=None)],
			outputs=[
				DataNode(id=input, dtype=Float, shape=[4, 4, 100, 100])],)
		OpNode(id=context_layer.1, op=aten::matmul, flops=10240000, mem_bytes=1049600, 
			inputs=[
				DataNode(id=attention_probs, dtype=Float, shape=[4, 4, 100, 100])
				DataNode(id=value_layer, dtype=Float, shape=[4, 4, 100, 32])],
			outputs=[
				DataNode(id=context_layer.1, dtype=Float, shape=[4, 4, 100, 32])],)


In [13]:
model_name = "prajjwal1/bert-tiny"
information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(information)

BertEmbeddings 1
<class 'dict'> embeddings:BertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

BertSelfAttention 2
<class 'tuple'> encoder.layer.0.attention.self:BertSelfAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

BertSelfOutput 2
<class 'tuple'> encoder.layer.0.attention.output:BertSelfOutput [torch.Size([4, 100, 128]), torch.Size([4, 100, 128])]

BertAttention 2
<class 'dict'> encoder.layer.0.attention:BertAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:BertAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None]

BertIntermediate 2
<class 'tuple'> encoder.layer.0.intermediate:BertIntermediate [torch.Size([4, 100, 128])]

BertOutput 2
<class 'tuple'> encoder.layer.0.output:BertOutput [torch.Size([4, 100, 512]), torch.Size([4, 100, 128])]

BertLayer

In [46]:
import torch
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from transformers import modeling_utils

def is_ml_operation(module):
    """
    This function checks if any given module is of a type that
    we want to analyse for E_ML operations
    """

    e_ml_operations = {nn.Linear, nn.LayerNorm, nn.Embedding, nn.BatchNorm1d,
                       nn.Conv1d, nn.MaxPool1d, nn.AvgPool1d, nn.LSTM, nn.Tanh,
                       modeling_utils.Conv1D}

    for e_ml_op in e_ml_operations:
        if isinstance(module, e_ml_op):
            return True
    return False

config = AutoConfig.from_pretrained(model_name)
config.torchscript = True
model=AutoModel.from_config(config).eval()

In [94]:
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torchscript": true,
  "transformers_version": "4.2.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [16]:
for name, module in model.named_modules():
    if not name:
        continue
    if not is_ml_operation(module):
        print(f'{name}:{module.__class__.__name__}')
#         print(f'\t{module}\n')

embeddings:BertEmbeddings
embeddings.dropout:Dropout
encoder:BertEncoder
encoder.layer:ModuleList
encoder.layer.0:BertLayer
encoder.layer.0.attention:BertAttention
encoder.layer.0.attention.self:BertSelfAttention
encoder.layer.0.attention.self.dropout:Dropout
encoder.layer.0.attention.output:BertSelfOutput
encoder.layer.0.attention.output.dropout:Dropout
encoder.layer.0.intermediate:BertIntermediate
encoder.layer.0.output:BertOutput
encoder.layer.0.output.dropout:Dropout
encoder.layer.1:BertLayer
encoder.layer.1.attention:BertAttention
encoder.layer.1.attention.self:BertSelfAttention
encoder.layer.1.attention.self.dropout:Dropout
encoder.layer.1.attention.output:BertSelfOutput
encoder.layer.1.attention.output.dropout:Dropout
encoder.layer.1.intermediate:BertIntermediate
encoder.layer.1.output:BertOutput
encoder.layer.1.output.dropout:Dropout
pooler:BertPooler


In [88]:

def get_non_parametric_ml_ops(model, input_ids):
    non_param_scopes = {name for name, module in model.named_modules() if not is_ml_operation(module)}
    trace = torch.jit.trace(model, input_ids)
    trace_graph = trace.inlined_graph
    graph, op_data_types = construct_aggregation_graph(trace_graph, fname)
    sigs = set()
    for node in graph.nodes:
        if node.op == 'aten::softmax':
            sig = f'op=softmax,flops={node.flops},mem_bytes={node.mem_bytes}'
            if sig not in sigs:
                print(node)
                sigs.add(sig)
        if node.op == 'aten::matmul' and node.scope in non_param_scopes:
            sig = f'op=matmul,flops={node.flops},mem_bytes={node.mem_bytes}'
            if sig not in sigs:
                print(node)
                sigs.add(sig)

In [102]:
from cg.node import construct_aggregation_graph
from calibrate_e_ml import get_non_parametric_ml_ops
model_name = "prajjwal1/bert-tiny"
# "google/mobilebert-uncased" "bert-base-uncased" "distilbert-base-uncased" "roberta-base" "prajjwal1/bert-tiny"
config = AutoConfig.from_pretrained(model_name)
config.torchscript = True
model=AutoModel.from_config(config).eval()
np_info = get_non_parametric_ml_ops(model, input_ids)

In [103]:
print_info(np_info)

matmul 4
<class 'tuple'> encoder.layer.0.attention.self [torch.Size([4, 2, 100, 64]), torch.Size([4, 2, 64, 100])]

softmax 2
<class 'tuple'> encoder.layer.0.attention.self [torch.Size([4, 2, 100, 100])]



In [83]:
for node in trace_graph.nodes():
    if node.kind() == 'prim::Constant':
#         print(node)
        print(node.outputs().toIValue())

0
1
tensor(0)
1
6
None
cpu
False
4
None
cpu
False
0
0
9223372036854775807
1
1
2
3
0
9223372036854775807
1
6
False
False
None
1.0
1
tensor(-10000., dtype=torch.float64)
0.1
128
1e-12
True
-1
False
9223372036854775807
0
tensor(0)
1
True
1e-12
1
0
2
64
3
-1
-2
tensor(8., dtype=torch.float64)
None
False
0.1
128
1
9223372036854775807
0


In [66]:
config = AutoConfig.from_pretrained("google/mobilebert-uncased")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:192: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, :seq_length]
/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:526: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(1000),


		OpNode(id=attention_scores.1, op=aten::matmul, flops=10240000, mem_bytes=1049600, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=query_layer.1, dtype=Float, shape=[4, 4, 100, 32])
				DataNode(id=236, dtype=Float, shape=[4, 4, 32, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 4, 100, 100])],)
		OpNode(id=input.10, op=aten::softmax, flops=800000, mem_bytes=1920000, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=input.9, dtype=Float, shape=[4, 4, 100, 100])
				DataNode(id=105, dtype=int)
				DataNode(id=108, dtype=None)],
			outputs=[
				DataNode(id=input.10, dtype=Float, shape=[4, 4, 100, 100])],)


In [67]:
config = AutoConfig.from_pretrained("bert-base-uncased")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=query_layer.1, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=172, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.7, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=input.6, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=100, dtype=int)
				DataNode(id=103, dtype=None)],
			outputs=[
				DataNode(id=input.7, dtype=Float, shape=[4, 12, 100, 100])],)


In [68]:
config = AutoConfig.from_pretrained("distilbert-base-uncased")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

		OpNode(id=scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, scope=transformer.layer.0.attention, 
			inputs=[
				DataNode(id=q.2, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=110, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.5, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=transformer.layer.0.attention, 
			inputs=[
				DataNode(id=input.4, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=58, dtype=int)
				DataNode(id=51, dtype=None)],
			outputs=[
				DataNode(id=input.5, dtype=Float, shape=[4, 12, 100, 100])],)


In [69]:
config = AutoConfig.from_pretrained("roberta-base")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=query_layer.1, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=178, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.7, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=input.6, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=106, dtype=int)
				DataNode(id=109, dtype=None)],
			outputs=[
				DataNode(id=input.7, dtype=Float, shape=[4, 12, 100, 100])],)


In [1]:
import bisect
import json
import pandas as pd
import numpy as np
from pathlib import Path
from common import sanitize

pd.set_option('display.float_format', '{:.6f}'.format)
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [2]:
res_names = ['cpu', 'mem', 'gpu', 'gpu_mem']

feature_names = ['batch_size', 'seq_len', 'flops',
                 'mem_bytes'] + \
                ['times_mean', 'times_std',
                 'gpu_power_mean', 'gpu_power_std',
                 'energy_mean', 'energy_std',
                 'level_name', 'model_name'] + res_names + [f'{k}_std' for k in res_names] 

def process_record(energy, prof_info, res, feature_values,
                   model_name, bs, runs, seq_len):
    res_np = res.to_numpy()
    res_t = res_np[:, 0]
    energy_np = energy.to_numpy()
    energy_t = energy_np[:, 0]
    for prof_item in prof_info:
        gpu_power_runs = []
        energy_runs = []
        times_runs = []
        res_runs = {k: [] for k in res_names}
        repeats = prof_item['repeats']
        for r in range(1, runs + 1):
            start_r = prof_item[f'start_{r}']
            end_r = prof_item[f'end_{r}']
            times_runs.append(end_r - start_r)

            res_s = bisect.bisect_right(res_t, start_r)
            res_e = bisect.bisect_right(res_t, end_r)
            res_r = res[res_s:res_e]
            for rn in res_names:
                res_runs[rn].append(res_r[rn].mean())
            gpu_power_r = res_r['gpu_power'].sum()
            gpu_power_runs.append(gpu_power_r)

            e_s = bisect.bisect_right(energy_t, start_r)
            e_e = bisect.bisect_right(energy_t, end_r)
            energy_r = energy[e_s:e_e]['value'].div(repeats).sum()
            energy_runs.append(energy_r)

        times_mean = np.mean(times_runs)
        times_std = np.std(times_runs) / times_mean * 100
        gpu_power_mean = np.mean(gpu_power_runs)
        gpu_power_std = np.std(gpu_power_runs) / gpu_power_mean * 100
        energy_mean = np.mean(energy_runs)
        energy_std = np.std(energy_runs) / energy_mean * 100
        for rn in res_names:
            feature_values[rn].append(np.mean(res_runs[rn]))
            rn_std = np.std(res_runs[rn]) / np.mean(res_runs[rn]) * 100
            feature_values[f'{rn}_std'].append(rn_std)

        flops = prof_item['flops'] / 1e6
        mem_bytes = prof_item['mem_bytes'] / 1024 / 1024
        feature_values['batch_size'].append(bs)
        feature_values['seq_len'].append(seq_len)
        feature_values['energy_mean'].append(energy_mean)
        feature_values['energy_std'].append(energy_std)
        feature_values['gpu_power_mean'].append(gpu_power_mean)
        feature_values['gpu_power_std'].append(gpu_power_std)
        feature_values['flops'].append(flops)
        feature_values['mem_bytes'].append(mem_bytes)
        feature_values['times_mean'].append(times_mean)
        feature_values['times_std'].append(times_std)
        feature_values['level_name'].append(prof_item['name'])
        feature_values['model_name'].append(model_name)


In [11]:
args=Namespace(out_dir='data/small-exp', 
               models=["google/mobilebert-uncased", "bert-base-uncased", "distilbert-base-uncased", "roberta-base"],
               exp_type='ml', exp_name='mlexp2',
               runs=10, batch_size=40, batch_step=8, seq_step=32, input_length=544)

out_dir = Path(args.out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
runs = args.runs
input_length = args.input_length
batch_size = args.batch_size
batch_step = args.batch_step
seq_step = args.seq_step
exp_type = args.exp_type
exp_name = args.exp_name

In [28]:
res = pd.read_csv(out_dir / f'{exp_name}-res.csv')
energy = pd.read_csv(out_dir / f'{exp_name}-energy.csv',
                     error_bad_lines=False, usecols=[0, 2])
energy = energy[energy['value'].apply(lambda x: is_float(x))]
energy = energy[energy['timestamp'].apply(lambda x: is_float(x))]

energy['value'] = energy['value'].astype(float).div(100)
energy['timestamp'] = energy['timestamp'].astype(float)
infos = []
for model_name in args.models:
    name_s = sanitize(model_name)
    feature_file = out_dir / f'{exp_name}_{name_s}_features.csv'
    feature_values = {k: [] for k in feature_names}
    for bs in list(range(8, batch_size, batch_step)):
        for seq_len in range(32, input_length, seq_step):
            
            filename = f'{name_s}_{exp_type}_r{runs}_b{bs}_i{seq_len}.json'
            prof_file = Path(out_dir) / exp_name / filename
            if not prof_file.exists():
                print(f'{prof_file} not exist!')
                continue
            with open(prof_file) as f:
                prof_info = json.load(f)
            process_record(energy, prof_info, res, feature_values,
                           model_name, bs, runs, seq_len)
    info = pd.DataFrame(data=feature_values)
    infos.append(info)
    info.to_csv(feature_file)
    print(f'{model_name} done.')

<ipython-input-2-8fc70e7b2ad8>:48: RuntimeWarning: invalid value encountered in double_scalars
  rn_std = np.std(res_runs[rn]) / np.mean(res_runs[rn]) * 100


google/mobilebert-uncased done.
bert-base-uncased done.
distilbert-base-uncased done.
roberta-base done.


In [7]:
res

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
0,1611101166.626549,1.100000,5.000000,2,0,0,14720,0,0,34522
1,1611101166.796815,0.600000,5.000000,2,0,0,14825,0,0,34473
2,1611101166.967060,0.100000,5.000000,2,0,0,14630,0,0,34570
3,1611101167.137330,0.100000,5.000000,2,0,0,14726,0,0,34473
4,1611101167.307614,0.100000,5.000000,2,0,0,14825,0,0,34570
...,...,...,...,...,...,...,...,...,...,...
312716,1611154402.833895,0.100000,5.000000,2,0,0,8396,0,0,6008
312717,1611154403.004164,1.100000,5.000000,2,0,0,8494,0,0,6154
312718,1611154403.174428,0.100000,5.000000,2,0,0,8490,0,0,6249
312719,1611154403.344674,0.600000,5.000000,2,0,0,8492,0,0,6249


In [15]:
for info in infos:
    print(info.shape)

(448, 20)
(64, 20)
(320, 20)
(64, 20)


In [17]:
infos[1]

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
0,8,32,0.000000,0.093994,0.390114,10.061582,99687.900000,19.070302,0.027893,3.804483,embeddings.position_embeddings:Embedding,bert-base-uncased,7.321667,11.000000,7.483333,0.200000,9.942225,0.000000,108.604930,300.000000
1,8,64,0.000000,0.187988,0.396814,9.876111,113639.400000,19.212641,0.032223,16.578860,embeddings.position_embeddings:Embedding,bert-base-uncased,7.263333,11.000000,9.233333,0.450000,12.763097,0.000000,109.999354,300.000000
2,8,96,0.000000,0.281982,0.408087,9.163128,127849.600000,18.979407,0.034123,18.758146,embeddings.position_embeddings:Embedding,bert-base-uncased,7.160000,11.000000,10.700000,0.750000,16.483077,0.000000,106.428133,300.000000
3,8,128,0.000000,0.375977,0.380339,10.142732,125164.100000,18.273739,0.028487,26.565901,embeddings.position_embeddings:Embedding,bert-base-uncased,6.726667,11.000000,11.900000,0.900000,14.436812,0.000000,95.700804,300.000000
4,8,160,0.000000,0.469971,0.374682,14.391839,124632.900000,16.261459,0.030248,20.017994,embeddings.position_embeddings:Embedding,bert-base-uncased,7.030000,11.000000,15.616667,1.050000,15.833553,0.000000,91.204718,300.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,32,384,0.000000,1.127930,0.396759,10.551210,201587.300000,17.360342,0.038488,14.872588,embeddings.position_embeddings:Embedding,bert-base-uncased,7.225000,11.000000,24.200000,5.666667,14.714180,0.000000,83.611652,182.148842
60,32,416,0.000000,1.221924,0.392991,9.387470,218736.100000,17.914643,0.040399,13.179194,embeddings.position_embeddings:Embedding,bert-base-uncased,7.230000,11.000000,29.433333,10.733333,14.858074,0.000000,66.533553,104.426361
61,32,448,0.000000,1.315918,0.387889,10.084984,223854.500000,16.222069,0.045902,20.619385,embeddings.position_embeddings:Embedding,bert-base-uncased,7.148333,11.000000,30.266667,12.216667,12.424077,0.000000,58.677914,80.805108
62,32,480,0.000000,1.409912,0.402399,10.100118,234802.300000,17.635323,0.044473,25.428520,embeddings.position_embeddings:Embedding,bert-base-uncased,7.078333,11.000000,31.416667,12.916667,13.102706,0.000000,53.484826,69.763984


In [296]:
infos[2].dropna().sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
26,32,96,267218.583552,9009.046875,8.508397,11.000000,97.403333,47.084286,0.716761,0.000000,0.834511,0.939147,6.105381,0.013890,7149497.000000,1.185902,110.030480,0.314705,distilbert-base-uncased,distilbert-base-uncased
14,16,224,320375.881728,7684.554688,8.452039,13.000000,97.748019,47.471304,0.504157,0.000000,1.115519,1.075000,7.692327,0.005014,8812926.600000,0.802637,135.957960,0.416252,distilbert-base-uncased,distilbert-base-uncased
23,24,256,552910.454784,12888.093750,8.606737,13.000000,98.812987,49.563337,0.742626,0.000000,0.779265,0.760980,13.130971,0.009864,15239726.800000,0.306798,233.934260,0.433762,distilbert-base-uncased,distilbert-base-uncased
31,32,256,737213.939712,17184.125000,8.544642,12.000000,99.243914,50.349796,0.378460,0.000000,0.340231,0.349927,17.350395,0.111740,20302028.800000,0.678733,311.653340,0.481888,distilbert-base-uncased,distilbert-base-uncased
29,32,192,545521.139712,13644.093750,9.037067,11.000000,98.573333,49.106667,0.767233,0.000000,0.705705,0.713012,12.735601,0.014315,14981904.600000,0.190907,230.381380,0.494689,distilbert-base-uncased,distilbert-base-uncased
30,32,224,640751.763456,15369.109375,8.886067,13.000000,99.485393,48.773034,2.062584,0.000000,0.157697,0.195151,15.209210,0.008151,17743795.200000,0.433082,273.227680,0.502665,distilbert-base-uncased,distilbert-base-uncased
27,32,128,358754.549760,10464.062500,8.464745,13.000000,98.413095,48.198129,0.378008,0.000000,0.730264,0.692230,8.268359,0.041995,9619756.000000,0.876812,146.896620,0.683801,distilbert-base-uncased,distilbert-base-uncased
11,16,128,179377.274880,5232.031250,8.446462,13.000000,96.554769,44.796923,1.774962,0.000000,1.795926,1.835574,4.342638,0.024293,4821553.200000,1.785010,74.930640,0.792158,distilbert-base-uncased,distilbert-base-uncased
18,24,96,200413.937664,6756.785156,8.520000,13.000000,97.184127,45.711111,1.362019,0.000000,1.173821,1.208262,4.739821,0.017427,5361371.000000,1.757455,82.880320,0.877253,distilbert-base-uncased,distilbert-base-uncased
21,24,192,409140.854784,10233.070312,8.465064,13.000000,98.553842,47.498911,0.558718,0.000000,0.725364,0.773401,9.722713,0.008940,11198226.800000,0.517100,171.861860,0.890473,distilbert-base-uncased,distilbert-base-uncased


In [297]:
infos[3].dropna().sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
31,32,256,1474943.909888,38283.125000,8.508525,13.000000,99.541730,55.186016,0.348381,0.000000,0.168512,0.257035,36.452941,0.074229,42826452.600000,0.136187,659.394280,0.176440,roberta-base,roberta-base
14,16,224,640946.862080,16831.150391,8.491075,13.000000,99.160896,52.015162,0.468395,0.000000,0.296474,0.388250,16.102221,0.021939,18519307.400000,0.488594,286.260440,0.239087,roberta-base,roberta-base
30,32,224,1281893.724160,33660.050781,9.029523,13.000000,99.527074,52.851741,1.815107,0.000000,0.279158,0.351909,31.849768,0.063903,37259910.200000,0.296302,574.288680,0.249235,roberta-base,roberta-base
27,32,128,717635.043328,21734.828125,8.517996,13.000000,99.205743,50.748020,0.346976,0.000000,0.220200,0.265863,17.132208,0.144713,19914179.000000,0.490498,307.390720,0.287794,roberta-base,roberta-base
29,32,192,1091325.517824,29360.976562,9.179919,12.000000,99.297950,52.945688,2.702689,0.000000,0.436990,0.448791,26.582488,0.073688,31182663.800000,0.225302,481.171700,0.302256,roberta-base,roberta-base
28,32,160,903239.290880,25385.902344,8.965966,13.000000,99.338792,51.254306,1.464205,0.000000,0.289965,0.406308,21.762289,0.147602,25667877.400000,0.288475,394.860140,0.315285,roberta-base,roberta-base
15,16,256,737471.954944,19142.687500,8.511865,13.000000,99.085371,53.058215,0.267594,0.000000,0.381918,0.403914,18.593617,0.040337,21241280.400000,0.196262,330.010140,0.322031,roberta-base,roberta-base
21,24,192,818494.138368,22021.294922,8.505210,13.000000,99.267227,51.275630,0.252548,0.000000,0.310596,0.301194,20.275331,0.038197,23352421.200000,0.179585,360.348560,0.356916,roberta-base,roberta-base
10,16,96,267256.387584,9205.001953,8.521000,13.000000,98.691667,46.388718,1.069904,0.000000,0.625956,0.619118,6.719930,0.162062,7381716.200000,0.804158,115.028300,0.382395,roberta-base,roberta-base
26,32,96,534512.775168,18407.753906,8.521351,12.000000,98.735135,49.356757,0.862931,0.000000,0.583160,0.595233,12.603253,0.155676,14743108.800000,0.408438,226.760560,0.382748,roberta-base,roberta-base
